## Creating a Model to Identify Handwritten Letters
This project aims to create a model that can identify handwritten text using the TensorFlow Library

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

### Load and Preprocess the Dataset

In [ ]:
IMAGE_SIZE = (32, 32)
DATA_DIR = './Desktop/Sci-Tech/Epoch/datasets/alphabets_dataset/alphabet_images'
CSV_FILE = './Desktop/Sci-Tech/Epoch/datasets/alphabets_dataset/alphabet_labels.csv'

df = pd.read_csv(CSV_FILE)

def load_and_preprocess_image(file_path):
    img = load_img(file_path, target_size=IMAGE_SIZE, color_mode='grayscale')
    img_array = img_to_array(img)
    img_array /= 255.0
    return img_array

images = []
labels = []
for index, row in df.iterrows():
    file_path = os.path.join(DATA_DIR, row['file'])
    img_array = load_and_preprocess_image(file_path)
    images.append(img_array)
    labels.append(row['label'])

images = np.array(images)
labels = np.array(labels)

# A-Z for 0-25
label_mapping = {chr(i + 65): i for i in range(26)}
labels = np.array([label_mapping[label] for label in labels])

labels = to_categorical(labels, num_classes=26)


### Split Dataset

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


### Build the model

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(26, activation='softmax')
])

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\Aroshish\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Train the Model

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val))


Epoch 1/20
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 150s 32ms/step - accuracy: 0.9815 - loss: 0.0628 - val_accuracy: 0.9913 - val_loss: 0.0331
Epoch 2/20
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 147s 32ms/step - accuracy: 0.9815 - loss: 0.0620 - val_accuracy: 0.9915 - val_loss: 0.0331
Epoch 3/20
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 148s 32ms/step - accuracy: 0.9818 - loss: 0.0618 - val_accuracy: 0.9912 - val_loss: 0.0323
Epoch 4/20
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 153s 33ms/step - accuracy: 0.9822 - loss: 0.0613 - val_accuracy: 0.9916 - val_loss: 0.0308
Epoch 5/20
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 210s 35ms/step - accuracy: 0.9824 - loss: 0.0604 - val_accuracy: 0.9911 - val_loss: 0.0329
Epoch 6/20
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 167s 36ms/step - accuracy: 0.9821 - loss: 0.0603 - val_accuracy: 0.9919 - val_loss: 0.0312
Epoch 7/20
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 174s 37ms/step - accuracy: 0.9824 - loss: 0.0603 - val_accuracy: 0.9920 - val_loss: 0.0301
Epoch 8/20
4656/4656 ━━━━━━━━━━━━━━━━━━━━ 178s 38ms/step - accuracy: 

### Evaulation Metrics

In [ ]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


2328/2328 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.9919 - loss: 0.0291
Validation Loss: 0.028448715806007385
Validation Accuracy: 0.9924017786979675


### Save the Model

In [ ]:
model.save('handwritten_text_recognition_model.h5')